In [1]:
import os, json, time
from pathlib import Path
from flask import Flask, request
from dotenv import load_dotenv
from stravalib import Client
import requests

In [8]:
load_dotenv()

CLIENT_ID = int(os.getenv("STRAVA_CLIENT_ID"))
CLIENT_SECRET = os.getenv("STRAVA_CLIENT_SECRET")
REDIRECT_URI = os.getenv("STRAVA_REDIRECT_URI", "http://127.0.0.1:5000/callback")
SCOPES = os.getenv("STRAVA_SCOPES", "read").split(",")

In [10]:
TOKENS_PATH = Path("strava_tokens.json")
API_BASE = "https://www.strava.com/api/v3"

app = Flask(__name__) #Creates the Flask app instance

#Function to save tokens in a json file
def save_tokens(t: dict):
    # stravalib returns expires_at (epoch), access_token, refresh_token, athlete info
    TOKENS_PATH.write_text(json.dumps(t, indent=2))


#Function to laod tokens from a json file
def load_tokens():
    return json.loads(TOKENS_PATH.read_text()) if TOKENS_PATH.exists() else None


#===================================
# Builds a stravalib.Client. If tokens are present and expired, stravalib can refresh them automatically 
# (since it knows your refresh token and expiry).
#===================================
def make_client():
    t = load_tokens() or {}
    return Client(
        access_token=t.get("access_token"),
        refresh_token=t.get("refresh_token"),
        token_expires=t.get("expires_at")
    )

#=======================
#This function will build the Strava authorization link and return a tiny HTML page.
#=======================
@app.route("/")
def index():
    url = Client().authorization_url(
        client_id=CLIENT_ID, redirect_uri=REDIRECT_URI, scope=SCOPES
    )
    return f"""
            <html>
            <body style="font-family:sans-serif">
                <h3>Strava OAuth</h3>
                <p><a href="{url}"><button>Connect Strava</button></a></p>
            </body>
            </html>
            """
#=====================
# Part the receive strava response
#=====================
@app.route("/callback")
def callback():
    if request.args.get("error"):
        return f"Error: {request.args['error']}", 400
    code = request.args.get("code") #Extracts the temporary authorization code from the URL.
    c = Client()
    token = c.exchange_code_for_token(
        client_id=CLIENT_ID, client_secret=CLIENT_SECRET, code=code
    )
    save_tokens(token)
    return "<h3>Authorized!</h3><p>You can close this tab and re-run the script.</p>"


#==================
#This function lets you call Strava's API directly, bypassing stravalib.
#So you can see the real JSON Strava returns — the raw API response.
#==================
def raw_get(path, params=None):
    """Direct REST call to see the exact JSON Strava returns."""
    t = load_tokens()
    if not t: raise RuntimeError("No tokens yet.")
    headers = {"Authorization": f"Bearer {t['access_token']}"} #This is how API requests prove your identity.
    r = requests.get(f"{API_BASE}{path}", headers=headers, params=params or {}, timeout=30)
    print(f'r status: {r.status_code}')
    if r.status_code == 401:
        print('Token expired, refreshing...')
        # Refresh the token using stravalib
        c = Client()
        refreshed_token = c.refresh_access_token(
            client_id=CLIENT_ID,
            client_secret=CLIENT_SECRET,
            refresh_token=t['refresh_token']
        )
        # Save the new tokens
        save_tokens(refreshed_token)
        # Retry the request with new token
        headers = {"Authorization": f"Bearer {refreshed_token['access_token']}"}
        r = requests.get(f"{API_BASE}{path}", headers=headers, params=params or {}, timeout=30)
    print('siamo qui')
    r.raise_for_status()
    return r.json(), r

if __name__ == "__main__":
    if not TOKENS_PATH.exists():
        print("Open http://127.0.0.1:5000 to authorize…")
        app.run("127.0.0.1", 5000, debug=False)
    else:
        client = make_client()

        # 1) Who am I?
        me = client.get_athlete()
        print(f"👋 Athlete: {me.firstname} {me.lastname} — id={me.id}")

        # 2) Show 5 most recent activities (friendly summary from stravalib objects)
        acts = list(client.get_activities(limit=5))
        print("\n📄 Recent activities (summary):")
        for i, a in enumerate(acts, 1):
            dist_km = (a.distance if getattr(a, "distance", None) else 0) / 1000.0
            move_s = int(a.moving_time) if a.moving_time else 0
            print(f"{i:2d}. {a.name} | {a.sport_type} | {dist_km:.2f} km | {move_s//60} min | {a.start_date_local:%Y-%m-%d}")

        # 3) Raw JSON (exact API format) for the most recent activity
        if acts:
            first_id = acts[0].id
            detail_json, _ = raw_get(f"/activities/{first_id}", params={"include_all_efforts": "true"})
            print("\n🧪 Raw JSON for the latest activity (first 1):")
            print(json.dumps(detail_json, indent=2)[:4000])  # avoid flooding the console

        # 4) Or: raw JSON list of recent activities (summary objects)
        summary_json, resp = raw_get("/athlete/activities", params={"per_page": 3, "page": 1})
        print("\n🧪 Raw JSON for 3 recent activities (summary objects):")
        print(json.dumps(summary_json, indent=2))

        # 5) Rate-limit headers (useful while testing)
        lim = resp.headers.get("X-RateLimit-Limit")
        use = resp.headers.get("X-RateLimit-Usage")
        if lim or use:
            print(f"\n⏳ Rate limit: {lim} | Usage: {use}")

👋 Athlete: Jacopo Signo — id=124721229

📄 Recent activities (summary):
 1. Lunch Run | root='Run' | 8.05 km | 44 min | 2025-11-11
 2. Ripetute fallimentari | root='Run' | 6.38 km | 41 min | 2025-11-07
 3. 3 + 2 (4.50km/Min) + 3 | root='Run' | 8.02 km | 43 min | 2025-11-04
 4. Workout | root='Run' | 2.53 km | 46 min | 2025-11-03
 5. Morning Run | root='Run' | 13.30 km | 74 min | 2025-11-02
r status: 200
siamo qui

🧪 Raw JSON for the latest activity (first 1):
{
  "resource_state": 3,
  "athlete": {
    "id": 124721229,
    "resource_state": 1
  },
  "name": "Lunch Run",
  "distance": 8047.5,
  "moving_time": 2698,
  "elapsed_time": 2698,
  "total_elevation_gain": 10.1,
  "type": "Run",
  "sport_type": "Run",
  "workout_type": null,
  "device_name": "Strava App",
  "id": 16423785338,
  "start_date": "2025-11-11T11:40:50Z",
  "start_date_local": "2025-11-11T12:40:50Z",
  "timezone": "(GMT+01:00) Europe/Rome",
  "utc_offset": 3600.0,
  "location_city": null,
  "location_state": null,
  "lo